In [1]:
import requests
import pandas as pd
import time

In [2]:
API_KEY = "" 
CHANNEL_ID = "UCW8Ews7tdKKkBT6GdtQaXvQ"

In [3]:
pageToken = ''
url = "https://www.googleapis.com/youtube/v3/search/?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000"+pageToken
response = requests.get(url).json()

In [4]:
#response
response['items'][0] # latest video

{'kind': 'youtube#searchResult',
 'etag': '6pXgw5-2qMai13Lja6-cpFCY1XQ',
 'id': {'kind': 'youtube#video', 'videoId': 'ToEl_tpyoM4'},
 'snippet': {'publishedAt': '2022-01-10T05:46:03Z',
  'channelId': 'UCW8Ews7tdKKkBT6GdtQaXvQ',
  'title': 'Uber Data Science Python Interview Question Walkthrough',
  'description': 'In this video, we will take a close look at a difficult data science interview question from Uber and walk you through a solution in ...',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/ToEl_tpyoM4/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/ToEl_tpyoM4/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/ToEl_tpyoM4/hqdefault.jpg',
    'width': 480,
    'height': 360}},
  'channelTitle': 'StrataScratch',
  'liveBroadcastContent': 'none',
  'publishTime': '2022-01-10T05:46:03Z'}}

In [5]:
video_Id = response['items'][0]["id"]['videoId']
video_title = response['items'][0]["snippet"]['title']
video_title = str(video_title).replace('&amp;','')

In [6]:
video_title

'Uber Data Science Python Interview Question Walkthrough'

In [7]:
video_date = response['items'][0]["snippet"]['publishedAt']
video_date = str(video_date).split("T")[0]
video_date

'2022-01-10'

In [8]:
for video in response['items']:
    if video['id']['kind'] == 'youtube#video' : 
        video_id = video['id']['videoId']
        video_title = video["snippet"]['title']
        video_title = str(video_title).replace('&amp;','')
        upload_date = video["snippet"]['publishedAt']
        upload_date = str(video_date).split("T")[0]
        
        print(video_id)
        print(video_title)
        print(upload_date)

ToEl_tpyoM4
Uber Data Science Python Interview Question Walkthrough
2022-01-10
30hS-MjpU6E
SQL Syntax Best Practices: How to Structure Your SQL Code
2022-01-10
Bgpp99iz0I0
Most Common Coding Mistakes on Data Science Interviews
2022-01-10
GeJUvdkJKEc
Multiple Solutions to Data Scientist Interview Question From Amazon [Rolling Average]
2022-01-10
GGURenNfXI0
Multiple Approaches for Solving a Twitter Data Scientist Interview Question
2022-01-10
PlpUo6bHsBQ
Advanced Facebook Data Science SQL interview question [RANK()]
2022-01-10
uY2wfR8Dkqo
5 Tips for a Successful Data Science Interview
2022-01-10
yY7yau9j3xk
Data Science SQL Scenario Based Interview Question [Facebook] - Date Manipulation
2022-01-10
i-E4pdU2qXM
Data Science SQL Interview Question Walkthrough [Microsoft] - Window Function: Ranking
2022-01-10
T1UhSuKqy3A
Most Common Data Science SQL Interview Question from DoorDash [window functions  partitions]
2022-01-10
2iE3JgNTwVU
A Walkthrough of Data Science SQL Interview Question fr

In [9]:
def get_video_details(video_id):

    #collecting view, like, dislike, comment counts
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
    response_video_stats = requests.get(url_video_stats).json()

    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    favorite_count = response_video_stats['items'][0]['statistics']['favoriteCount']
    comment_count = response_video_stats['items'][0]['statistics']['commentCount']

    return view_count, like_count, favorite_count, comment_count

In [10]:
def get_videos(df):
    pageToken = ""
    while 1:
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken

        response = requests.get(url).json()
        time.sleep(1) #give it a second before starting the for loop
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&amp;","")
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]
                view_count, like_count, favorite_count, comment_count = get_video_details(video_id)

                df = df.append({'video_id':video_id,'video_title':video_title,
                                "upload_date":upload_date,"view_count":view_count,
                                "like_count":like_count,"favorite_count":favorite_count,
                                "comment_count":comment_count},ignore_index=True)
        try:
            if response['nextPageToken'] != None: #if none, it means it reached the last page and break out of it
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            break


    return df

In [11]:
df2 = pd.DataFrame(columns=["video_id","video_title","upload_date","view_count","like_count","favorite_count","comment_count"]) 

df2 = get_videos(df2)

In [12]:
df2

,video_id,video_title,upload_date,view_count,like_count,favorite_count,comment_count
0,ToEl_tpyoM4,Uber Data Science Python Interview Question Wa...,2022-01-10,7896,183,0,19
1,30hS-MjpU6E,SQL Syntax Best Practices: How to Structure Yo...,2021-12-22,6257,308,0,31
2,Bgpp99iz0I0,Most Common Coding Mistakes on Data Science In...,2021-11-04,3443,171,0,29
3,GeJUvdkJKEc,Multiple Solutions to Data Scientist Interview...,2021-10-11,4726,143,0,32
4,GGURenNfXI0,Multiple Approaches for Solving a Twitter Data...,2021-09-13,3591,150,0,32
...,...,...,...,...,...,...,...
75,UX4_IgagL9I,How to Use Google Colaboratory | Import a CSV ...,2020-05-02,1772,12,0,2
76,tDdo3FiWpgE,Interview Questions for SQL Joins and Subqueries,2020-02-01,1013,14,0,0
77,wW827gqxlRY,SQL Job Interview Mistakes #2,2019-09-15,925,20,0,0
78,xbc2GpGUXwc,SQL Job Interview Mistakes #1,2019-08-03,7535,65,0,1


In [13]:
df2[df2["video_title"].str.contains("SQL")]

,video_id,video_title,upload_date,view_count,like_count,favorite_count,comment_count
1,30hS-MjpU6E,SQL Syntax Best Practices: How to Structure Yo...,2021-12-22,6257,308,0,31
5,PlpUo6bHsBQ,Advanced Facebook Data Science SQL interview q...,2021-08-09,12020,389,0,51
7,yY7yau9j3xk,Data Science SQL Scenario Based Interview Ques...,2021-07-20,4915,138,0,22
8,i-E4pdU2qXM,Data Science SQL Interview Question Walkthroug...,2021-07-13,4494,175,0,19
9,T1UhSuKqy3A,Most Common Data Science SQL Interview Questio...,2021-06-15,7202,260,0,39
10,2iE3JgNTwVU,A Walkthrough of Data Science SQL Interview Qu...,2021-06-04,4021,136,0,30
14,xtbMCAVXDmU,Facebook and Microsoft Data Science SQL interv...,2021-04-15,6655,201,0,56
16,tNXliLTlrV8,Step-by-step Approach to Solving Any Data Scie...,2021-03-24,6785,224,0,32
17,XRwxYOhHdE8,Facebook&#39;s Most Common Data Science SQL In...,2021-03-10,21434,546,0,45
20,QenwDm5oWdU,Advanced Data Science SQL Interview Question [...,2021-02-17,13899,495,0,91


In [14]:
df2[df2["video_title"].str.contains("SQL")].shape

(28, 7)

### Find CSV files in directory

In [15]:
import os
import numpy as np
import pandas as pd

In [17]:
def csv_files():
    csv_files = []
    for file in os.listdir(os.getcwd()):
        if file.endswith('.csv'):
            csv_files.append(file)
    return csv_files

In [18]:
def configure_dataset_directory(csv_files, dataset_dir):
  
    #make dataset folder to process csv files
    try: 
        mkdir = 'mkdir {0}'.format(dataset_dir)
        os.system(mkdir)
    except:
        pass

    #move csv files to dataset folder
    for csv in csv_files:
        mv_file = "mv '{0}' {1}".format(csv, dataset_dir)
        os.system(mv_file)

    return

In [19]:
def create_df(dataset_dir, csv_files):
  
    data_path = os.getcwd()+'/'+dataset_dir+'/'

    #loop through the files and create the dataframe
    df = {}
    for file in csv_files:
        try:
            df[file] = pd.read_csv(data_path+file)
        except UnicodeDecodeError:
            df[file] = pd.read_csv(data_path+file, encoding="ISO-8859-1") #if utf-8 encoding error
        print(file)
    
    return df

In [20]:
!ls

extract_data_api.ipynb
